In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

tf.test.is_gpu_available()

2.0.0-dev20191002
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.17.2
pandas 0.25.1
sklearn 0.21.3
tensorflow 2.0.0-dev20191002
tensorflow_core.keras 2.2.4-tf
Instructions for updating:
Use `tf.config.experimental.list_physical_devices('GPU')` instead.


False

# 使用tf.function(func) 来讲py func 变成 tf graph

In [7]:
# tf.function and auto-graph
def scaled_elu(z,scale = 1.0,alpha = 1.0):
    """使用tf.function来实现selu"""
    # z > 0 ? scale * z : scale * alpha * tf.nn.elu(z)
    is_positive = tf.greater_equal(z,0.0)
#     print(is_positive) # tf.Tensor([False False], shape=(2,), dtype=bool)
    return scale * tf.where(is_positive, z, alpha * tf.nn.elu(z))

print(scaled_elu(tf.constant(-3.)))
print(scaled_elu(tf.constant([-3.,-2.5])))

# 将Python函数转换成tf的图结构函数：普通函数转换成图结构函数计算会快很多
scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant(-3.)))
print(scaled_elu_tf(tf.constant([-3.,-2.5])))

# 通过转化后的tf图结构函数召回它的python函数
print(scaled_elu_tf.python_function == scaled_elu)

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
True


## 变成tf.graph之后的函数会比py func 计算快

In [10]:
# 普通函数转换成图结构函数计算会快很多,我们来对比一下
%timeit scaled_elu(tf.random.normal((1000,1000)))
%timeit scaled_elu_tf(tf.random.normal((1000,1000)))

4.21 ms ± 96.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.06 ms ± 122 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# 使用@tf.function也可以将 py func 变为 tf graph

In [30]:
# 使用@tf.function 来把函数转换成tf的图结构函数
@tf.function # 在函数前加上这个注释就可以吧Python函数转换为tf的图结构函数
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total
print(converge_to_2(20))

tf.Tensor(1.9999981, shape=(), dtype=float32)


# 可以使用tf.autograph.to_code(func)将变换后的tf graph代码打印出来

In [24]:
# 将Python函数转换中中间图结构函数的中间代码展示出来
def display_tf_code(func): # 将普通的Python函数转换成tf的代码
    code = tf.autograph.to_code(func)
    from IPython.display import display, Markdown
    display(Markdown('```python\n{}\n```'.format(code))) # 用Markdown的样式将代码打印出来

In [25]:
display_tf_code(scaled_elu)

```python
def tf__scaled_elu(z, scale=None, alpha=None):
  """使用tf.function来实现selu"""
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('scaled_elu', 'scaled_elu_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as scaled_elu_scope:
    is_positive = ag__.converted_call(tf.greater_equal, scaled_elu_scope.callopts, (z, 0.0), None, scaled_elu_scope)
    do_return = True
    retval_ = scaled_elu_scope.mark_return_value(scale * ag__.converted_call(tf.where, scaled_elu_scope.callopts, (is_positive, z, alpha * ag__.converted_call(tf.nn.elu, scaled_elu_scope.callopts, (z,), None, scaled_elu_scope)), None, scaled_elu_scope))
  do_return,
  return ag__.retval(retval_)

```

In [32]:
display_tf_code(converge_to_2.python_function) # 因为converge_to_2是tf的func所以要转成Python的func

```python
def tf__converge_to_2(n_iters):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('converge_to_2', 'converge_to_2_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as converge_to_2_scope:
    total = ag__.converted_call(tf.constant, converge_to_2_scope.callopts, (0.0,), None, converge_to_2_scope)
    increment = ag__.converted_call(tf.constant, converge_to_2_scope.callopts, (1.0,), None, converge_to_2_scope)

    def get_state():
      return ()

    def set_state(_):
      pass

    def loop_body(iterates, increment, total):
      _ = iterates
      total += increment
      increment /= 2.0
      return increment, total
    increment, total = ag__.for_stmt(ag__.converted_call(range, converge_to_2_scope.callopts, (n_iters,), None, converge_to_2_scope), None, loop_body, get_state, set_state, (increment, total), ('increment', 'total'), ())
    do_return = True
    retval_ = converge_to_2_scope.mark_return_value(total)
  do_return,
  return ag__.retval(retval_)

```

# 变量tf.Variable 不能在tf graph函数内部定义，只能在函数外部定义，然后在函数内部进行值修改

In [37]:
# 变量是不能放在tf函数的内部去定义的
var = tf.Variable(0.) # 在func外部去定义变量
@tf.function
def add_21():
    return var.assign_add(21) # 在内部去修改变量的值；asssign_add相当于+=，但是不能直接用+=
print(add_21())

tf.Tensor(21.0, shape=(), dtype=float32)


# 使用input_signature可以定义tf graph接受的参数类型

In [42]:
# 使用input_signature可以定义函数接受的数据类型
@tf.function(input_signature=[tf.TensorSpec([None],tf.int32,name='x')])
def cube(z):
    return tf.pow(z,3)
try:
    print(cube(tf.constant([1.,2.,3.])))
except ValueError as ex:
    print(ex,'\n')
    
print(cube(tf.constant([1,2,3])))

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x')) 

tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


# 使用cube.get_concrete_function 可以添加input_signature —>SavedModel

In [45]:
# @tf.function : py func -> tf graph
# get_concrete_function -> add input signature ->SavedModel
cube_func_int32 = cube.get_concrete_function(
    tf.TensorSpec([None],tf.int32))
print(cube_func_int32)

In [47]:
print(cube_func_int32 is cube.get_concrete_function(
    tf.TensorSpec([5],tf.int32)))
print(cube_func_int32 is cube.get_concrete_function(
    tf.constant([1,2,3])))

True
True


In [48]:
cube_func_int32.graph

In [49]:
cube_func_int32.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [55]:
# 通过索引位置取出pow
pow_op = cube_func_int32.graph.get_operations()[2]
print(pow_op)

name: "Pow"
op: "Pow"
input: "x"
input: "Pow/y"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



In [56]:
# 通过名字取出pow
pow_op = cube_func_int32.graph.get_operation_by_name('Pow')
print(pow_op)

name: "Pow"
op: "Pow"
input: "x"
input: "Pow/y"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



In [53]:
print(list(pow_op.inputs))
print(list(pow_op.outputs))

[<tf.Tensor 'x:0' shape=(None,) dtype=int32>, <tf.Tensor 'Pow/y:0' shape=() dtype=int32>]
[<tf.Tensor 'Pow:0' shape=(None,) dtype=int32>]


In [57]:
cube_func_int32.graph.get_operation_by_name('x')

<tf.Operation 'x' type=Placeholder>

In [58]:
cube_func_int32.graph.get_tensor_by_name("x:0")

<tf.Tensor 'x:0' shape=(None,) dtype=int32>

In [59]:
cube_func_int32.graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "x"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 175
}

# 这些取Graph的操作在搭建训练模型时不会用，但是在模型Inference或者和模型结构转换成其他语言时是非常重要的